In [6]:
from openai import AzureOpenAI
import json
import os
import Helper
from Helper import get_positive_cases_for_state_on_date
from Helper import get_hospitalized_increase_for_state_on_date
from dotenv import load_dotenv
import time 
from IPython.display import clear_output

In [7]:
load_dotenv()

True

In [8]:
# III) Add message
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT")
)

In [9]:
assistant = client.beta.assistants.create(
    instructions="""You are an assistant answering questions 
                  about a Covid dataset.""",
    model=os.getenv('AZURE_DEPLOYMENT'),
    tools=Helper.tools_sql)


In [10]:
thread = client.beta.threads.create()
print(thread)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska
               the 2021-03-05?"""
)
print(message.model_dump_json(indent=2))

Thread(id='thread_004lyIxVtH9qWicgRqoZkYfv', created_at=1732724662, metadata={}, object='thread')
{
  "id": "msg_CO5H5jhC1gITeMRtLAtoMQag",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "how many hospitalized people we had in Alaska\n               the 2021-03-05?"
      },
      "type": "text"
    }
  ],
  "created_at": 1732724663,
  "file_ids": [],
  "metadata": {},
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_004lyIxVtH9qWicgRqoZkYfv"
}


In [12]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_CO5H5jhC1gITeMRtLAtoMQag",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska\n               the 2021-03-05?"
          },
          "type": "text"
        }
      ],
      "created_at": 1732724663,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_004lyIxVtH9qWicgRqoZkYfv"
    }
  ],
  "object": "list",
  "first_id": "msg_CO5H5jhC1gITeMRtLAtoMQag",
  "last_id": "msg_CO5H5jhC1gITeMRtLAtoMQag",
  "has_more": false
}


In [13]:
# IV) Run assistant on thread

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [15]:
start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id, run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
    )
    status = run.status
    print(f'Status: {status}')
    if (status == "requires_action"):
        available_functions = {
            "get_positive_cases_for_state_on_date": get_positive_cases_for_state_on_date,
            "get_hospitalized_increase_for_state_on_date": get_hospitalized_increase_for_state_on_date
        }

        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                state_abbr=function_args.get("state_abbr"),
                specific_date=function_args.get("specific_date"),
            )
            print(function_response)
            print(tool_call.id)
            tool_outputs.append(
                {"tool_call_id": tool_call.id,
                 "output": str(function_response)
                 }
            )

        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )

messages = client.beta.threads.messages.list(
    thread_id=thread.id
)



Elapsed time: 0 minutes 5 seconds
Status: in_progress
Elapsed time: 0 minutes 10 seconds
Status: failed


In [16]:
print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_yvN2hFEahHuHVhZldR9MzetC",
      "assistant_id": "asst_tOULvdBrrAqa681ZiB4JPnvY",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "To provide you with the accurate number of hospitalized people in Alaska on March 5, 2021, I need to check the available COVID-19 dataset for this specific information. Let me do that for you."
          },
          "type": "text"
        }
      ],
      "created_at": 1732724680,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_VNepCQZnfFF2mgSBA4IdRWs2",
      "thread_id": "thread_0XkdXZIAA40TgvOziZlFqHZ3"
    },
    {
      "id": "msg_7Kc7gGBJJ1ywJ1G8jzAIwGqm",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska in 2021-03-05?"
          },
          "type": "text"
   

In [20]:
data = open("./data/all-states-history.csv", "rb")
file = client.files.create(
    file=data,
    purpose='assistants'
)

assistant = client.beta.assistants.create(
    instructions="""You are an assistant answering questions about
                  a Covid dataset.""",
    model=os.getenv('AZURE_DEPLOYMENT'),
    tools=[{"type": "code_interpreter"}]
)

# Linking the file to the assistant through another method if available or manually managing the file within your process
# if no direct method exists, you can load the file data into the assistant’s context during interactions.

thread = client.beta.threads.create()

print(thread)

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska in 2021-03-05?"""
)

print(message)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


Thread(id='thread_FQJjkxMWkwCSC50wFo14x6As', created_at=1732724782, metadata={}, object='thread')
ThreadMessage(id='msg_DArhm4RIraXAWp2mVwYPyfxQ', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='how many hospitalized people we had in Alaska in 2021-03-05?'), type='text')], created_at=1732724782, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_FQJjkxMWkwCSC50wFo14x6As')


In [21]:
status = run.status
start_time = time.time()
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
    )
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)
    
if status == "failed":
    error_details = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Error Details:", error_details)

messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

Error Details: Run(id='run_bQrzv1ZDHuogmpSeBZEE1gdD', assistant_id='asst_g3rJftQTzqitgXBnxv8jkgQU', cancelled_at=None, completed_at=None, created_at=1732724782, expires_at=None, failed_at=1732724804, file_ids=[], instructions='You are an assistant answering questions about\n                  a Covid dataset.', last_error=LastError(code='rate_limit_exceeded', message='Rate limit is exceeded. Try again in 48 seconds.'), metadata={}, model='gpt4-turbo', object='thread.run', required_action=None, started_at=1732724783, status='failed', thread_id='thread_FQJjkxMWkwCSC50wFo14x6As', tools=[ToolAssistantToolsCode(type='code_interpreter')], usage=Usage(completion_tokens=0, prompt_tokens=0, total_tokens=0), temperature=1.0, top_p=1.0, max_completion_tokens=None, max_prompt_tokens=None, truncation_strategy={'type': 'auto', 'last_messages': None}, incomplete_details=None, response_format='auto', tool_choice='auto', parallel_tool_calls=True)


In [ ]:
print(messages.model_dump_json(indent=2))